In [1]:
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
spark = get_spark_session(dev=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/23 15:13:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_namespace = 'gold' 
# spark.sql(f"DROP NAMESPACE IF EXISTS {test_namespace} CASCADE")
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}").show()

++
||
++
++



In [3]:
TARGET_CATEGORY = 'health_beauty'

In [ ]:
health_beauty_sales_stats_bcg = spark.read.table('warehousedev.gold.health_beauty_sales_stats_bcg')
delivered_order_product = spark.read.table('warehousedev.gold.delivered_order_product')

+--------------------+--------------------+--------------------+-------------+--------------------+-----------+
|            order_id|          product_id|            category|order_item_id|           seller_id|total_price|
+--------------------+--------------------+--------------------+-------------+--------------------+-----------+
|00010242fe8c5a6d1...|4244733e06e7ecb49...|          cool_stuff|            1|48436dade18ac8b2b...|      72.19|
|00018f77f2f0320c5...|e5f2d52b802189ee6...|            pet_shop|            1|dd7ddc04e1b6c2c61...|     259.83|
|000229ec398224ef6...|c777355d18b72b67a...|     furniture_decor|            1|5b51032eddd242adc...|     216.87|
|00024acbcdf0a6daa...|7634da152a4610f15...|           perfumery|            1|9d7a1d34a50524090...|      25.78|
|00042b26cf59d7ce6...|ac6c3623068f30de0...|        garden_tools|            1|df560393f3a51e745...|     218.04|
|00048cc3ae777c65d...|ef92defde845ab845...|          housewares|            1|6426d21aca402a131...|     

In [6]:
delivered_order_product.show()


+--------------------+--------------------+--------------------+-------------+--------------------+-----------+
|            order_id|          product_id|            category|order_item_id|           seller_id|total_price|
+--------------------+--------------------+--------------------+-------------+--------------------+-----------+
|00010242fe8c5a6d1...|4244733e06e7ecb49...|          cool_stuff|            1|48436dade18ac8b2b...|      72.19|
|00018f77f2f0320c5...|e5f2d52b802189ee6...|            pet_shop|            1|dd7ddc04e1b6c2c61...|     259.83|
|000229ec398224ef6...|c777355d18b72b67a...|     furniture_decor|            1|5b51032eddd242adc...|     216.87|
|00024acbcdf0a6daa...|7634da152a4610f15...|           perfumery|            1|9d7a1d34a50524090...|      25.78|
|00042b26cf59d7ce6...|ac6c3623068f30de0...|        garden_tools|            1|df560393f3a51e745...|     218.04|
|00048cc3ae777c65d...|ef92defde845ab845...|          housewares|            1|6426d21aca402a131...|     

In [7]:
delivered_order_product_bcg = delivered_order_product.join(health_beauty_sales_stats_bcg.select('product_id', 'segment'), on='product_id', how='inner')
delivered_order_product_bcg.show(n=5)

+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------+
|          product_id|            order_id|     category|order_item_id|           seller_id|total_price|       segment|
+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------+
|310ae3c140ff94b03...|0005a1a1728c9d785...|health_beauty|            1|a416b6a846a117243...|      157.6|    Niche Gems|
|d63c1011f49d98b97...|00061f2a7bc09da83...|health_beauty|            1|cc419e0650a3c5ba7...|      68.87|Volume Drivers|
|e67307ff0f15ade43...|001d8f0e34a38c37f...|health_beauty|            1|f4aba7c0bca51484c...|      26.77|Volume Drivers|
|e67307ff0f15ade43...|001d8f0e34a38c37f...|health_beauty|            2|f4aba7c0bca51484c...|      26.77|Volume Drivers|
|bdcf6a834e8faa30d...|001e7cf2ad6bef3ad...|health_beauty|            1|2a84855fd20af891b...|       51.1|Volume Drivers|
+--------------------+------------------

In [8]:
delivered_order_product_bcg.writeTo(f"{test_namespace}.delivered_order_product_bcg").createOrReplace()

In [9]:
spark.stop()